In [1]:
import pandas as pd

# Считываем датасет
df = pd.read_csv("spam.csv", encoding='latin-1')[['v1','v2']]
df.columns = ['label','text']

# Преобразуем метку в бинарную
df['label'] = df['label'].map({'ham':0, 'spam':1})

# X/y
X = df['text']
y = df['label']

df.head()


,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Разделение на train/test

In [8]:

!pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp314-cp314-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 1.9 MB/s  0:00:04 eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


Бейзлайн (константная модель)

In [10]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

baseline = DummyClassifier(strategy='most_frequent')  # всегда предсказывает чаще встречающийся класс
baseline.fit(X_train, y_train)
y_pred_baseline = baseline.predict(X_test)

print("F1-score бейзлайна:", f1_score(y_test, y_pred_baseline))


F1-score бейзлайна: 0.0


Преобразование текста и обучение простой модели

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("F1-score логистической регрессии:", f1_score(y_test, y_pred))


F1-score логистической регрессии: 0.8847583643122676


Более сложная модель с подбором гиперпараметров

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier(random_state=42))
])

params = {
    'clf__n_estimators':[50,100],
    'clf__max_depth':[10,20,None]
}

grid = GridSearchCV(pipe_rf, params, cv=3, scoring='f1')
grid.fit(X_train, y_train)

y_pred_rf = grid.predict(X_test)
print("Лучшие параметры:", grid.best_params_)
print("F1-score RandomForest:", f1_score(y_test, y_pred_rf))


Лучшие параметры: {'clf__max_depth': None, 'clf__n_estimators': 50}
F1-score RandomForest: 0.8796992481203008


Интерпретация модели (SHAP для RandomForest)

In [14]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.6/555.6 kB 5.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 10.5 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 9.4 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [shap]5/6 [shap]]te]
